In [89]:
import pandas as pd


Ограничения на выгрузку записей справочника "Скважины" на стороне КССС. Скважина может быть выгружена, если в связанном справочнике "Связи Скважин и Компаний" есть хотя бы одна запись, у которой:
1) поле "Скважина"(WELL_ID) заполненно ссылкой на выгружаемую запись из справочника "Скважины" 
2) поле "Компания, получившая лицензию"(COMPANY_ID) заполнено ссылкой на запись из справочника "КОНТРАГЕНТЫ" или "Особые группы компаний", для которой есть запись в справочнике "Иерархии компаний по правилам формирования ИС" с заполненным полем "Квалификатор иерархии (дерева) организаций в ИС" (COMPANY_GROUP_SPECIAL_ID) = 5687138 (Иерархия предприятий УР OIS (УР OIS)).
Зависимая выгрузка справочника "Связи Скважин и Компаний" и справочника "Скважины":  
При создании записей справочника "Связи Скважин и Компаний", первыми на выгрузку идут связанные записи справочника "Скважины".

In [90]:
ier = pd.read_excel('Иерархия.xlsx')

In [91]:
ier_ois = ier[(ier['COMPANY_GROUP_SPECIAL_ID'] == '5687138') ]

In [92]:
skv = pd.read_excel('скважины.xlsx')

In [93]:
ogc = pd.read_excel('Связь скважин и компаний.xlsx')

In [95]:
for_ois = ier_ois['CONTRAGENT_ID']

In [96]:
ogc_ois = ogc[ogc['COMPANY_ID'].isin(for_ois)]

In [97]:
wells_for_ois = ogc_ois['WELL_ID']

In [98]:
skv_ois = skv[skv['WELL_ID'].isin(wells_for_ois)]

In [99]:
skv_ois

,WELL_ID,FIELD_ID,FIELD_NAME,FIELD_AREA_ID,FIELD_AREA_NAME,PROSPECT_ID,PROSPECT_NAME,BASIN_ID,BASIN_NAME,WELL_UWI,...,WELL_INV_NUMBER,LIFT_NUM,PS_WELL,BOREHOLE,NGDO,TPP,CITS,CDNG,START_DATE,END_DATE
1,1,15,"Ватьеганское, Западно-Сибирская НГП, Ханты-Ман...",1812965,Ватьеганская,782,"Западно-Сибирская НГП, Ватьеганская",240,Западно-Сибирская НГП,060317510_08039_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
2,100,15,"Ватьеганское, Западно-Сибирская НГП, Ханты-Ман...",1812965,Ватьеганская,782,"Западно-Сибирская НГП, Ватьеганская",240,Западно-Сибирская НГП,060317559_08452_,...,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
7,1000,249,"Усинское, Харьяга-Усинский НГР, Усинск",NaN,NaN,NaN,NaN,391,Харьяга-Усинский НГР,1400100513000220000,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
8,10000,305,"Южно-Ягунское, Западно-Сибирская НГП, Ханты-Ма...",1812959,Южно-Ягунская,1281,"Сургутский НГР, Южно-Ягунская",240,Западно-Сибирская НГП,067293336_05282_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
9,100000,444,"Юрчукское, Волго-Уральская НГП, Соликамский",1521900,Южно-Юрчукская,NaN,NaN,394,Волго-Уральская НГП,001900020001_000761,...,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121260,9995,305,"Южно-Ягунское, Западно-Сибирская НГП, Ханты-Ма...",1812959,Южно-Ягунская,1281,"Сургутский НГР, Южно-Ягунская",240,Западно-Сибирская НГП,067293336_05275_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
121261,9996,305,"Южно-Ягунское, Западно-Сибирская НГП, Ханты-Ма...",1812959,Южно-Ягунская,1281,"Сургутский НГР, Южно-Ягунская",240,Западно-Сибирская НГП,067293336_05276_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
121262,9997,305,"Южно-Ягунское, Западно-Сибирская НГП, Ханты-Ма...",1812959,Южно-Ягунская,1281,"Сургутский НГР, Южно-Ягунская",240,Западно-Сибирская НГП,067293336_05277_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
121274,9998,305,"Южно-Ягунское, Западно-Сибирская НГП, Ханты-Ма...",1812959,Южно-Ягунская,1281,"Сургутский НГР, Южно-Ягунская",240,Западно-Сибирская НГП,067293336_05279_,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
skv_ois.to_excel('output.xlsx', sheet_name='Sheet1', index=False)

In [117]:
kls = pd.read_excel('классы.xlsx')

In [119]:
sbl = pd.read_excel('шаблоны.xlsx')

In [120]:
result = pd.merge(kls,sbl, on = 'Hierarchy_MTR_Class', how = 'left')

In [121]:
result

,Hierarchy_MTR_Class,Hierarchy_MTR_Name,CSCD_ID,SHORT_NAME/ru_RU,SHORT_NAME/en_US,AUTO_SHORT_NAME,FULL_NAME/ru_RU,FULL_NAME/en_US,AUTO_FULL_NAME,FULL_NAME_NTD/ru_RU,...,MAIN_PRODUCT,BELONGING_TO_MAIN_PRODUCT,NO_SALE_MINOR,NPRD_DUTY_CATEGORY,NPRD_DUTY_CATEGORY_EXCEPTION,NPRD_DUTY_CATEGORY_ACTUAL,MP_ALLOWED,CTM,Item_creation_date,shablon
0,2004020101990000,Нефтегазосепараторы прочие,6043894,Сепаратор газа топливного 82-V-1020,NaN,Сепаратор газа топливного 82-V-1020,Сепаратор газа топливного 82-V-1020,NaN,Сепаратор газа топливного 82-V-1020,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
1,2004020101990000,Нефтегазосепараторы прочие,6043895,Сепаратор газа кислого 34-V-9002,NaN,Сепаратор газа кислого 34-V-9002,Сепаратор газа кислого 34-V-9002,NaN,Сепаратор газа кислого 34-V-9002,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
2,2004039900000000,Емкости прочие,6043932,Бункер разгрузочный 400 4м3,NaN,Бункер разгрузочный 400 4м3,Бункер разгрузочный 400 4м3,NaN,Бункер разгрузочный 400 4м3,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102221
3,2004020101990000,Нефтегазосепараторы прочие,6043896,Сепаратор газа кислого 34-V-9001,NaN,Сепаратор газа кислого 34-V-9001,Сепаратор газа кислого 34-V-9001,NaN,Сепаратор газа кислого 34-V-9001,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
4,2004020101990000,Нефтегазосепараторы прочие,6043897,Сепаратор газа факельного Е-1,NaN,Сепаратор газа факельного Е-1,Сепаратор газа факельного Е-1,NaN,Сепаратор газа факельного Е-1,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,2004020101990000,Нефтегазосепараторы прочие,6043891,Сепаратор газа 83-V-0306,NaN,Сепаратор газа 83-V-0306,Сепаратор газа 83-V-0306,NaN,Сепаратор газа 83-V-0306,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
525,2004039900000000,Емкости прочие,6043930,Емкость сбора конденсата Е-101 20м3,NaN,Емкость сбора конденсата Е-101 20м3,Емкость сбора конденсата Е-101 20м3,NaN,Емкость сбора конденсата Е-101 20м3,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102221
526,2004020101990000,Нефтегазосепараторы прочие,6043892,Сепаратор газа 83-V-0307,NaN,Сепаратор газа 83-V-0307,Сепаратор газа 83-V-0307,NaN,Сепаратор газа 83-V-0307,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220
527,2004020101990000,Нефтегазосепараторы прочие,6043893,Сепаратор газа хвостового V-602,NaN,Сепаратор газа хвостового V-602,Сепаратор газа хвостового V-602,NaN,Сепаратор газа хвостового V-602,NaN,...,False,NaN,False,NaN,NaN,NaN,NaN,False,21.04.2025,102220


In [124]:
cols = result.columns.to_list()
cols

['Hierarchy_MTR_Class',
 'Hierarchy_MTR_Name',
 'CSCD_ID',
 'SHORT_NAME/ru_RU',
 'SHORT_NAME/en_US',
 'AUTO_SHORT_NAME',
 'FULL_NAME/ru_RU',
 'FULL_NAME/en_US',
 'AUTO_FULL_NAME',
 'FULL_NAME_NTD/ru_RU',
 'FULL_NAME_NTD/en_US',
 'MODEL/ru_RU',
 'MODEL/en_US',
 'MATERIAL_MARK',
 'KEY_WORD/ru_RU',
 'KEY_WORD/en_US',
 'BASE_UNIT',
 'MSEHI',
 'MSEHL',
 'LUKOIL_MATERIAL',
 'TMC_Stat',
 'TMC_Type',
 'TMC_Kind',
 'TMC_Kind_Exception',
 'TMC_KIND_ACTUAL_NAME',
 'ACTUAL_RECORD',
 'GROUP_MATERIAL',
 'GROUP_MATERIAL_NAME',
 'EXCISE_GROUP',
 'EXCISE_GROUP_NAME',
 'CRUDE_OIL_MATERIALS',
 'EXCISE_GOODS_IN_CHECK',
 'PRODUCER',
 'PRODUCER_NAME',
 'LUKOIL_CODE',
 'REGULATORY_DOC_NAME',
 'MATERIAL_NTD',
 'TECH_DESCRIPTION',
 'Hierarchy_SBIT_Class',
 'Hierarchy_SBIT_Name',
 'Hierarchy_TNVED_Class',
 'Hierarchy_TNVED_Name',
 'TN_VED_OTHERS',
 'Hierarchy_OKPD2_Class',
 'Hierarchy_OKPD2_Name',
 'Hierarchy_OKVED2_Class',
 'Hierarchy_OKVED2_Name',
 'Hierarchy_UBM_Class',
 'Hierarchy_UBM_Name',
 'BRAND_OIL',
 

In [130]:
result2 = result[['Hierarchy_MTR_Class','shablon','CSCD_ID','SHORT_NAME/ru_RU','SHORT_NAME/en_US']]

In [131]:
result2

,Hierarchy_MTR_Class,shablon,CSCD_ID,SHORT_NAME/ru_RU,SHORT_NAME/en_US
0,2004020101990000,102220,6043894,Сепаратор газа топливного 82-V-1020,NaN
1,2004020101990000,102220,6043895,Сепаратор газа кислого 34-V-9002,NaN
2,2004039900000000,102221,6043932,Бункер разгрузочный 400 4м3,NaN
3,2004020101990000,102220,6043896,Сепаратор газа кислого 34-V-9001,NaN
4,2004020101990000,102220,6043897,Сепаратор газа факельного Е-1,NaN
...,...,...,...,...,...
524,2004020101990000,102220,6043891,Сепаратор газа 83-V-0306,NaN
525,2004039900000000,102221,6043930,Емкость сбора конденсата Е-101 20м3,NaN
526,2004020101990000,102220,6043892,Сепаратор газа 83-V-0307,NaN
527,2004020101990000,102220,6043893,Сепаратор газа хвостового V-602,NaN


In [138]:
import pandas as pd
from bs4 import BeautifulSoup

# Загрузка CSV
df = pd.read_excel('export.xlsx')

def parse_xml_column(xml_str):
    if pd.isna(xml_str) or xml_str.strip() == '':
        return {}

    soup = BeautifulSoup(xml_str, 'html.parser')

    result = {}
    # перебираем все строки таблицы
    for tr in soup.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) >= 2:
            # ключ — текст из <b> внутри первого <td>, без двоеточия
            key_tag = tds[0].find('b')
            if key_tag:
                key = key_tag.get_text(strip=True).rstrip(':')
                # значение — текст из второго <td>
                value = tds[1].get_text(strip=True)
                result[key] = value
    return result

# Разбор столбцов N34306 и N52205
for col in ['N34306', 'N52205']:
    parsed_data = df[col].apply(parse_xml_column)
    # превращаем dict-столбцы в датафреймы и добавляем с префиксом колонки
    parsed_df = pd.json_normalize(parsed_data).add_prefix(f'{col}_')
    df = pd.concat([df, parsed_df], axis=1)

# Можно удалить исходные колонки, если нужно
# df = df.drop(columns=['N34306', 'N52205'])

df.head(25)


c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,ALL_ID,KSSS_ID,N2889,N2864,N34306,N52205,N34306_Код КССС ТРУ,N34306_№ п/п ТРУ,N34306_Перечень ТРУ,N52205_Код КССС ТРУ,N52205_№ п/п ТРУ,N52205_Перечень ТРУ
0,3612207,3612207,False,False,<table style='border: 1px solid'><tr><td style...,NaN,832»4»611. Продовольственные товары,611.,NaN,NaN,NaN,NaN
1,3612207,3612207,False,False,<table style='border: 1px solid'><tr><td style...,NaN,2656»4»656. Продовольственные товары,656,NaN,NaN,NaN,NaN
2,3040308,3040308,False,False,<table style='border: 1px solid'><tr><td style...,NaN,832»4»611. Продовольственные товары,611.,NaN,NaN,NaN,NaN
3,3040308,3040308,False,False,<table style='border: 1px solid'><tr><td style...,NaN,2656»4»656. Продовольственные товары,656,NaN,NaN,NaN,NaN
4,2377029,2377029,False,False,<table style='border: 1px solid'><tr><td style...,NaN,2656»4»656. Продовольственные товары,656,NaN,NaN,NaN,NaN
5,6095555,2377233,False,False,<table style='border: 1px solid'><tr><td style...,NaN,2523»1»523. Специальная одежда,523,NaN,NaN,NaN,NaN
6,6095556,2377235,False,True,<table style='border: 1px solid'><tr><td style...,NaN,2523»1»523. Специальная одежда,523,NaN,NaN,NaN,NaN
7,2129430,2129430,False,False,<table style='border: 1px solid'><tr><td style...,NaN,779»1»574. Спецодежда,574.,1</td,NaN,NaN,NaN
8,2129430,2129430,False,False,<table style='border: 1px solid'><tr><td style...,<table style='border: 1px solid'><tr><td style...,2523»1»523. Специальная одежда,523,NaN,2523»1»523. Специальная одежда,523,NaN
9,2377018,2377018,False,False,<table style='border: 1px solid'><tr><td style...,NaN,2656»4»656. Продовольственные товары,656,NaN,NaN,NaN,NaN


## Парсинг шаблонов для 

In [5]:
import os
import pandas as pd
from openpyxl import load_workbook

# Укажите путь к папке с файлами
folder_path = 'Данные'

# Итоговый датафрейм
result_df = pd.DataFrame(columns=['Файл', 'Значение Краткое наименование','Значение Полное наименование'])

for root, dirs, files in os.walk(folder_path):  # перебор всех подпапок
    for file in files:
        if file.endswith(('.xlsx', '.xlsm', '.xls')):
            file_path = os.path.join(root, file)
            try:
                wb = load_workbook(file_path, read_only=True, data_only=True)
                
                # Ищем лист с подстрокой "Эталонные данные" в названии
                sheet_name = None
                for sheet in wb.sheetnames:
                    if "Эталонные записи" in sheet:
                        sheet_name = sheet
                        break
                
                if sheet_name is None:
                    print(f'В файле {file} лист с "Эталонные записи" не найден. Пропускаем.')
                    continue
                
                ws = wb[sheet_name]
                
                # Находим строку с "Краткое наименование нормализованное" в 3-м столбце (столбец C)
                found_row = None
                for row in ws.iter_rows(min_col=3, max_col=3):
                    cell = row[0]
                    if cell.value == "Краткое наименование нормализованное":
                        found_row = cell.row
                        break
                
                if found_row is None:
                    print(f'В листе {sheet_name} файла {file} не найдена ячейка с "Краткое наименование нормализованное". Пропускаем.')
                    continue
                
                # Собираем все значения ниже найденной ячейки в столбце 3
                values = []
                current_row = found_row + 2
                while True:
                    cell_value = ws.cell(row=current_row, column=3).value
                    if cell_value is None:
                        break
                    values.append(cell_value)
                    current_row += 1

                # Находим строку с "Полное наименование нормализованное" в 5-м столбце (столбец C)
                found_row = None
                for row in ws.iter_rows(min_col=5, max_col=5):
                    cell = row[0]
                    if cell.value == "Полное наименование нормализованное":
                        found_row = cell.row
                        break
                
                if found_row is None:
                    print(f'В листе {sheet_name} файла {file} не найдена ячейка с "Полное наименование нормализованное". Пропускаем.')
                    continue
                
                # Собираем все значения ниже найденной ячейки в столбце 3
                values_full = []
                current_row = found_row + 2
                while True:
                    cell_value = ws.cell(row=current_row, column=5).value
                    if cell_value is None:
                        break
                    values_full.append(cell_value)
                    current_row += 1
                
                # Добавляем данные в итоговый датафрейм
                df_temp = pd.DataFrame({
                    'Файл': [file]*len(values),
                    'Значение Краткое наименование': values,
                    'Значение Полное наименование': values_full
                })
                result_df = pd.concat([result_df, df_temp], ignore_index=True)
            except Exception as e:
                print(f'Ошибка в файле {file}: {e}')

# Сохраняем итоговый датафрейм в Excel
output_path = os.path.join(folder_path, 'собранные_данные2.xlsx')
result_df.to_excel(output_path, index=False)
print(f'Сбор данных завершён. Результат сохранён в {output_path}')



В файле StandartnyjotcetvEXCELTMCiuslugi_1751543289099.xlsx лист с "Эталонные записи" не найден. Пропускаем.


c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


В файле TMCiuslugi_12.07.2025 10.42.52.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле StandartnyjotcetvEXCELTMCiuslugi_1751550737220.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле StandartnyjotcetvEXCELTMCiuslugi_1752672014801.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_07.07.2025 16.14.29.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_16.07.2025 16.44.34.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_07.07.2025 15.11.59.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_18.06.2025 09.54.45.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_01.07.2025 13.25.21.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_03.07.2025 09.52.31.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_12.07.2025 10.48.00.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_01.07.2025 13.06.10

In [9]:
import os
import pandas as pd
from openpyxl import load_workbook

# Укажите путь к папке с файлами
folder_path = 'Данные'

# Итоговый датафрейм
result_df = pd.DataFrame(columns=['Файл', 'Значение Краткое наименование', 'Значение Полное наименование'])

# Список для хранения сообщений об ошибках
log_messages = []

for root, dirs, files in os.walk(folder_path):  # перебор всех подпапок
    for file in files:
        if file.endswith(('.xlsx', '.xlsm', '.xls')):
            file_path = os.path.join(root, file)
            try:
                wb = load_workbook(file_path, read_only=True, data_only=True)

                # Ищем лист с подстрокой "Эталонные записи" в названии
                sheet_name = None
                for sheet in wb.sheetnames:
                    if "Эталонные записи" in sheet:
                        sheet_name = sheet
                        break

                if sheet_name is None:
                    msg = f'В файле {file} лист с "Эталонные записи" не найден. Пропускаем.'
                    print(msg)
                    log_messages.append(msg)
                    continue

                ws = wb[sheet_name]

                # Находим строку с "Краткое наименование нормализованное" в 3-м столбце (столбец C)
                found_row = None
                for row in ws.iter_rows(min_col=3, max_col=3):
                    cell = row[0]
                    if cell.value == "Краткое наименование нормализованное":
                        found_row = cell.row
                        break

                if found_row is None:
                    msg = f'В листе {sheet_name} файла {file} не найдена ячейка с "Краткое наименование нормализованное". Пропускаем.'
                    print(msg)
                    log_messages.append(msg)
                    continue

                values = []
                current_row = found_row + 2
                while True:
                    cell_value = ws.cell(row=current_row, column=3).value
                    if cell_value is None:
                        if current_row == found_row + 2:
                            msg = f'В листе {sheet_name} файла {file} нет записей "Краткое наименование нормализованное".'
                            print(msg)
                            log_messages.append(msg)
                        break
                    values.append(cell_value)
                    current_row += 1

                # Находим строку с "Полное наименование нормализованное" в 5-м столбце (столбец E)
                found_row = None
                for row in ws.iter_rows(min_col=5, max_col=5):
                    cell = row[0]
                    if cell.value == "Полное наименование нормализованное":
                        found_row = cell.row
                        break

                if found_row is None:
                    msg = f'В листе {sheet_name} файла {file} не найдена ячейка с "Полное наименование нормализованное". Пропускаем.'
                    print(msg)
                    log_messages.append(msg)
                    continue

                values_full = []
                current_row = found_row + 2
                while True:
                    cell_value = ws.cell(row=current_row, column=5).value
                    if cell_value is None:
                        if current_row == found_row + 2:
                            msg = f'В листе {sheet_name} файла {file} нет записей "Полное наименование нормализованное".'
                            print(msg)
                            log_messages.append(msg)    
                        break
                    values_full.append(cell_value)
                    current_row += 1

                # Проверяем, чтобы длина списков совпадала
                min_len = min(len(values), len(values_full))
                if min_len == 0:
                    msg = f'В файле {file} нет данных для "Краткое наименование" или "Полное наименование". Пропускаем.'
                    print(msg)
                    log_messages.append(msg)
                    continue

                # Формируем итоговый датафрейм
                df_temp = pd.DataFrame({
                    'Файл': [file] * min_len,
                    'Значение Краткое наименование': values[:min_len],
                    'Значение Полное наименование': values_full[:min_len]
                })
                result_df = pd.concat([result_df, df_temp], ignore_index=True)

            except Exception as e:
                msg = f'Ошибка при обработке файла {file}: {e}'
                print(msg)
                log_messages.append(msg)

# Сохраняем итоговый датафрейм в Excel
output_path = os.path.join(folder_path, 'собранные_данные.xlsx')
result_df.to_excel(output_path, index=False)
print(f'Сбор данных завершён. Результат сохранён в {output_path}')

# Сохраняем логи в отдельный Excel файл
if log_messages:
    log_df = pd.DataFrame({'Сообщения': log_messages})
    log_path = os.path.join(folder_path, 'логи_обработки.xlsx')
    log_df.to_excel(log_path, index=False)
    print(f'Логи обработки сохранены в {log_path}')
else:
    print('Не обнаружено ошибок и предупреждений.')


В файле StandartnyjotcetvEXCELTMCiuslugi_1751543289099.xlsx лист с "Эталонные записи" не найден. Пропускаем.


c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


В файле TMCiuslugi_12.07.2025 10.42.52.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле StandartnyjotcetvEXCELTMCiuslugi_1751550737220.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле StandartnyjotcetvEXCELTMCiuslugi_1752672014801.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_07.07.2025 16.14.29.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_16.07.2025 16.44.34.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_07.07.2025 15.11.59.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_18.06.2025 09.54.45.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_01.07.2025 13.25.21.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_03.07.2025 09.52.31.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_12.07.2025 10.48.00.xlsx лист с "Эталонные записи" не найден. Пропускаем.
В файле TMCiuslugi_01.07.2025 13.06.10